In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import json
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
from ipywidgets import IntProgress
from IPython.display import display
import os

In [4]:
url_base= 'https://www.lojasrenner.com.br/'

In [25]:
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')

fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')


def createDriver():
    
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'    

    chrome_options = Options()  
    chrome_options.add_argument("--headless")  
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument(f'user-agent={user_agent}')
    chrome_options.add_argument("--window-size=1825x9000")

    return webdriver.Chrome('/usr/bin/chromedriver', options = chrome_options)

browser = createDriver()
browser.get(url_base)


In [26]:
#Clickear en FEMENINO
browser.find_element_by_link_text("MODA ÍNTIMA").click()
url_femenino = browser.current_url

In [27]:
#CLICK EN CATEGORIA ***
boton_categoria = WebDriverWait(browser, 50).until(EC.element_to_be_clickable((By.LINK_TEXT,'CATEGORIA')))
boton_categoria.click()

In [28]:
#Elementos para generar la lista de categorias
elements = browser.find_elements_by_xpath('//*[@id="item0"]/div[2]/div')

In [29]:
#Lista de categorias
categorias = []
for i in elements:
    categorias.append(i.text.split('\n'))
categorias = categorias[0]
#Se remueven las excepciones

['Bolsas e Calçados', 'Feminino', 'Masculino', 'Moda Íntima']

In [31]:
excepts = ['Bolsas e Calçados','Feminino','Masculino']
categorias = [ x for x in categorias if x not in excepts ]

In [32]:
try:
    browser.switch_to.frame("social-push")
    browser.execute_script('document.querySelector("#btClose").click()')
    browser.switch_to.default_content()
except:
    pass

In [33]:
browser.find_element_by_link_text(categorias[0]).click()

In [35]:
url_moda_intima = browser.current_url

In [36]:
time.sleep(1.5)
categorias_int = browser.find_elements_by_xpath('//*[@id="item0"]/div[2]/div')

In [38]:
for intc in categorias_int:
    print(intc.text)

Acessórios Sensuais
Body
Calcinhas
Corset e Camisetes
Camisolas
Cintas Liga
Meias
Meias-Calças
Modeladores e Redutores
Pijamas e Short Dolls
Robes e Roupões
Sutiãs
Monte seu Conjunto
Pantufas


In [39]:
CATEGORIAS = []
for intc in categorias_int:
    CATEGORIAS.append(intc.text.split('\n'))

In [40]:
CATEGORIAS

[['Acessórios Sensuais',
  'Body',
  'Calcinhas',
  'Corset e Camisetes',
  'Camisolas',
  'Cintas Liga',
  'Meias',
  'Meias-Calças',
  'Modeladores e Redutores',
  'Pijamas e Short Dolls',
  'Robes e Roupões',
  'Sutiãs',
  'Monte seu Conjunto',
  'Pantufas']]

In [41]:
CATEGORIAS = CATEGORIAS[0]
excepts = ['Calçados','Curve & Plus Size','Acessórios Sensuais','Meias-Calças','Meias','Cintas Liga','Pantufas']

for item in excepts:
    try:
        CATEGORIAS.remove(item)
    except:
        pass

In [42]:
#CATEGORIAS
#['Blusas e Camisetas', 'Calças', 'Casacos e Jaquetas']
categorias_link = []
for item in CATEGORIAS:
    browser.find_element_by_link_text(item).click()
    categorias_link.append([item,browser.current_url])
    
    browser.get(url_moda_intima)
    boton_categoria = WebDriverWait(browser, 50).until(EC.element_to_be_clickable((By.LINK_TEXT,'CATEGORIA1')))
    boton_categoria.click()

In [44]:
#PAGINACION EN 'cantidad_hojas'
#26/07/ 10.52 a.m
#Clickea en categorias
cantidad_hojas = []
for catlink in categorias_link:
    try:
        browser.get(catlink[1])
        wait = WebDriverWait(browser, 50)
    except:
        browser.quit()
        time.sleep(1)
        browser = createDriver()
        continue

#    CLICK EN EL BOTON COLOR 
#    browser.execute_script('''
#    document.getElementsByClassName('filter_item')[3].getElementsByClassName('handle cp')[0].click();
#    ''')
    
    #html_colores = browser.execute_script('''var x = document.getElementsByClassName('df items')[2].textContent;return x;''')
    html_colores = browser.find_elements(By.CSS_SELECTOR,'.df.items')[2].find_elements_by_tag_name('a')
    colors = [i.get_attribute('text').strip() for i in html_colores]

    for item_color in colors:
        

        #CLICK EN EL COLOR DE LA LISTA
        time.sleep(4)
        browser.execute_script(f'''$("a.item.cp span.npe:contains('{item_color}')").click();''')
        
        lastPage = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'js-lastPage'))).get_attribute('data-last-page')
        lastPage = int(lastPage)

        for cant in range(lastPage):
            
            cantidad_hojas.append([catlink[0],item_color,lastPage,browser.current_url])
            try:
                wait.until(EC.presence_of_element_located((By.CLASS_NAME,'js-next-paginator'))).click()
                
                
            except:
                pass
        
        #click en el boton activo
        browser.execute_script('window.scrollTo(0,0);')
        time.sleep(3)
        try:
            
            browser.execute_script(f'''$("a.item.cp span.npe:contains('{item_color}')").click();''')

            #browser.execute_script(f'''$("a.item.cp span.npe:contains('{item_color}')").click();''')
            
            time.sleep(1)
        except:
            pass
        

In [45]:
cantidad_hojas

[['Body',
  'Vermelho',
  1,
  'https://www.lojasrenner.com.br/lista/moda-intima/moda-intima/body/-/N-1ixpb34Z4g3hcdZ1hwylc0Zlqf5mw/p1'],
 ['Body',
  'Bege',
  1,
  'https://www.lojasrenner.com.br/lista/moda-intima/moda-intima/body/-/N-1ixpb34Zivs9dxZ1hwylc0Zlqf5mw/p1'],
 ['Body',
  'Branco',
  1,
  'https://www.lojasrenner.com.br/lista/moda-intima/moda-intima/body/-/N-1ixpb34Z112vlz5Z1hwylc0Zlqf5mw/p1'],
 ['Body',
  'Preto',
  1,
  'https://www.lojasrenner.com.br/lista/moda-intima/moda-intima/body/-/N-1ixpb34Zsp16waZ1hwylc0Zlqf5mw/p1'],
 ['Calcinhas',
  'Rosa',
  7,
  'https://www.lojasrenner.com.br/lista/moda-intima/moda-intima/calcinhas/-/N-1xf3iueZ14rldr9Z1hwylc0Zlqf5mw/p1'],
 ['Calcinhas',
  'Rosa',
  7,
  'https://www.lojasrenner.com.br/lista/moda-intima/moda-intima/calcinhas/-/N-1xf3iueZ14rldr9Z1hwylc0Zlqf5mw/p1'],
 ['Calcinhas',
  'Rosa',
  7,
  'https://www.lojasrenner.com.br/lista/moda-intima/moda-intima/calcinhas/-/N-1xf3iueZ14rldr9Z1hwylc0Zlqf5mw/p1'],
 ['Calcinhas',
  'Ros

In [46]:
# CATEGORIA, COLOR, NUMERO, LINK
rennerLista = []
for ITEM in cantidad_hojas:
    precio = []
    
    browser.get(ITEM[3])
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    page = BeautifulSoup(browser.page_source,'html.parser')
    try:
        articulos = [i.find_all(class_='item_product') for i in page.find_all(class_='wrapper cf results-list js-results-list')][0]
    except:
        #SIGNIFICA QUE NO SE ENCONTRARON RESULTADOS, PAGINA VACIA
        continue
    
    
    browser.execute_script("window.scrollTo(0, 2000);")
    time.sleep(0.8)
    browser.execute_script("window.scrollTo(0, 4000);")
    time.sleep(0.8)
    browser.execute_script("window.scrollTo(0, 5600);")
    time.sleep(0.8)
    #element_price = elemento 
    element_price = browser.find_elements_by_class_name('price_product')
    #moverse en el dom para sacar el atributo span class price
    prices = [ene.find_elements_by_class_name('price')[0].text for ene in element_price]

    #SI SPLIT ES 2 SIGNIFICA QUE NO TIENE PRECIO CON DESCUENTO
    #ELSE  SIGNIFICA QUE TIENE PRECIO CON DESCUENTO
    for item in prices:
        if len(item.split()) ==2:
            precio.append([item.split()[1],item.split()[1]])
        else:
            try:
                precio.append([item.split()[1],item.split()[3]])
            except:
                precio.append(['99999','99999'])
            
    indice_aux = 0
    for articulo in articulos:
        data = json.loads(articulo['data-product-gtm'])
        #Imagen del producto
        try:
            img = [i.img['src'] for i in articulo.find_all(class_='img_product')][0]
        except:
            img = [None]
            
        
        rennerLista.append([data['id'],ITEM[0],ITEM[1],data['name'],precio[indice_aux][0],precio[indice_aux][1],data['brand'],data['url'],img])
        indice_aux += 1  

In [47]:
dfRenner = pd.DataFrame(rennerLista)

In [48]:
#dfRenner.head(1)
dfRenner = dfRenner.rename(columns={0: 'id Producto'})
dfRenner = dfRenner.rename(columns={1: 'Tipo'})
dfRenner = dfRenner.rename(columns={2: 'Color'})
dfRenner = dfRenner.rename(columns={3: 'Descripcion'})
dfRenner = dfRenner.rename(columns={4: 'Precio Original'})
dfRenner = dfRenner.rename(columns={5: 'Precio'})
dfRenner = dfRenner.rename(columns={6: 'Marca'})
dfRenner = dfRenner.rename(columns={7: 'Url Producto'})
dfRenner = dfRenner.rename(columns={8: 'Url Imagen'})

In [49]:
dfRenner['Fecha'] = hoy

In [50]:
hoy

'2019/12/19'

In [51]:
dfRenner = dfRenner.drop_duplicates()

In [52]:
dfRenner = dfRenner.reset_index(drop=True)

In [53]:
len(dfRenner)

1721

In [54]:
#dfRenner["Precio Anterior"] = (dfrenner["Precio Anterior"]
dfRenner['Precio Original'] = (dfRenner['Precio Original'].str.extract(r"([\d,\.]+)", expand=False)
                               .str.replace(",", ".")
                               .astype(float))

In [55]:
dfRenner['Precio'] = (dfRenner['Precio'].str.extract(r"([\d,\.]+)", expand=False)
                               .str.replace(",", ".")
                               .astype(float))

In [56]:
writer = ExcelWriter('./Salida/dfRenner_moda_intima'+str(fecha)+'.xlsx')
dfRenner.to_excel(writer,'Hoja1')
writer.save()

In [57]:
browser.quit()

In [58]:
len(dfRenner.dropna())

1721

In [59]:
len(dfRenner)

1721